In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build
import exafmm as fmm
import exafmm.laplace as laplace
import exafmm.helmholtz as helmholtz
import numpy as np

In [2]:
print(fmm.__doc__)
print(laplace.__doc__)
print(helmholtz.__doc__)

exafmm's pybind11 module
exafmm's submodule for Laplace kernel
exafmm's submodule for Helmholtz kernel


#### create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

In [4]:
src_coords = np.random.random((nsrcs, 3))
src_charges = np.random.random(nsrcs)
trg_coords = np.random.random((ntrgs, 3))

sources_l = laplace.init_sources(src_coords, src_charges)
targets_l = laplace.init_targets(trg_coords)

In [5]:
src_charges = np.zeros(nsrcs, dtype=np.complex_)
src_charges.real = np.random.random(nsrcs)
src_charges.imag = np.random.random(nsrcs)

sources_h = helmholtz.init_sources(src_coords, src_charges)
targets_h = helmholtz.init_targets(trg_coords)

#### creat FMM instances, set FMM parameters

In [6]:
fmm_l = laplace.LaplaceFMM()
fmm_l.p = 10
fmm_l.nsurf = 6*(fmm_l.p-1)*(fmm_l.p-1) + 2
fmm_l.ncrit = 400
fmm_l.depth = 4

In [7]:
fmm_h = helmholtz.HelmholtzFMM()
fmm_h.p = 10
fmm_h.nsurf = 6*(fmm_h.p-1)*(fmm_h.p-1) + 2
fmm_h.ncrit = 400
fmm_h.depth = 4
fmm_h.wavek = 10

#### build tree

In [8]:
tree_l = laplace.build_tree(sources_l, targets_l, fmm_l)

In [9]:
print(fmm_l.r0)  # root node radius
print(fmm_l.x0)  # root node coords

0.5000034543989219
[0.500000, 0.500002, 0.499995]


In [10]:
tree_l.leafs[5].level

4

In [11]:
tree_h = helmholtz.build_tree(sources_h, targets_h, fmm_h)

In [12]:
print(fmm_h.r0)  # root node radius
print(fmm_h.x0)  # root node coords

0.5000034543989219
[0.500000, 0.500002, 0.499995]


#### build interaction lists

In [13]:
fmm.init_rel_coord()  # initialize relative coordinates matrix

In [14]:
skip_P2P = False   # flag to skip near-field evaluation
laplace.build_list(tree_l, fmm_l, skip_P2P)
helmholtz.build_list(tree_h, fmm_h, skip_P2P)

#### M2L setup

In [15]:
fmm_l.M2L_setup(tree_l.nonleafs)

In [16]:
fmm_h.M2L_setup(tree_h.nonleafs)

#### pre-compute

In [17]:
fmm_l.precompute()

In [18]:
fmm_h.precompute()

#### evaluate potential

In [19]:
laplace.evaluate(tree_l, fmm_l)

P2M                  : 1.5419500e-01
M2M                  : 8.2932000e-02
P2L                  : 1.9720000e-03
M2P                  : 3.3800000e-03
P2P                  : 4.4868000e-02
M2L                  : 6.5462500e-01
L2L                  : 8.2195000e-02
L2P                  : 1.3196800e-01


array([8223.42716133, 8679.79682574, 8285.86883738, ..., 7636.36893991,
       8479.72362367, 7423.11940284])

In [20]:
helmholtz.evaluate(tree_h, fmm_h)

P2M                  : 9.0571800e-01
M2M                  : 3.8199500e-01
P2L                  : 1.7190000e-03
M2P                  : 1.9140000e-03
P2P                  : 4.4791300e-01
M2L                  : 1.7198400e+00
L2L                  : 3.3937700e-01
L2P                  : 9.3614600e-01


array([ -525.02481241-480.18510919j,  -128.14269288-801.55364624j,
        -309.52925307-723.12094992j, ...,  -696.82761699-384.51397617j,
        -775.7565601 -313.00048039j, -1072.55718272+180.9451959j ])

#### check accuracy

For now, this is only valid when skip_P2P is disabled.

In [21]:
fmm_l.verify(tree_l.leafs)

[5.367274254865698e-10, 9.509989287074475e-08]

In [22]:
fmm_h.verify(tree_h.leafs)

[3.251556200080978e-08, 7.345734896567816e-07]

#### update charges and run FMM iteratively

niters = 10
for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    src_charges = np.random.random(nsrcs)   # generate new random charges
    fmm.update(src_charges)      # update charges
    fmm.clear()                  # clear values
    potentials = fmm.evaluate()  # evaluate potentials
    fmm.check_accuracy()         # check accuracy